In [25]:
import json
import snowflake.connector
from typing import List
from datetime import datetime

In [26]:
# Fully qualified table names
TABLE_1 = "FIVETRAN_RAW.SCOTTS.SCOTTS"
TABLE_2 = "_CLONE_DATAOPS_PENG_8343_SNOWPIPE_RAW.SCOTTS_PIPE.SCOTTS"

In [27]:
EXCLUDED_COLUMNS = ['METADATA_FILENAME', 'METADATA_FILE_ROW_NUMBER', '_DBT_COPIED_AT', '_FILE', '_FIVETRAN_SYNCED', '_LINE', '_MODIFIED']

In [28]:
# Generate log file path with timestamp
def create_log_file():
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return f"compare_log_{timestamp}.txt"

In [29]:
# Write both to console and log file
class Logger:
    def __init__(self, filepath):
        self.file = open(filepath, "w", encoding="utf-8")

    def log(self, message):
        print(message)
        self.file.write(message + "\n")

    def close(self):
        self.file.close()

In [30]:
# Load config from external JSON file
def load_config(path='config.json'):
    with open(path, 'r') as f:
        return json.load(f)

In [31]:
# Connect using externalbrowser SSO
def connect_to_snowflake(config):
    return snowflake.connector.connect(
        user=config["user"],
        account=config["account"],
        warehouse=config["warehouse"],
        authenticator="externalbrowser"
    )

In [32]:
# Get column names for a table
def get_columns(cursor, table_name: str, excluded: List[str]) -> List[str]:
    cursor.execute(f"DESC TABLE {table_name}")
    return [row[0] for row in cursor.fetchall() if row[0] not in excluded]

In [33]:
# Count distinct values per column
def get_distinct_counts(cursor, table_name: str, columns: List[str]) -> dict:
    counts = {}
    for col in columns:
        query = f'SELECT COUNT(DISTINCT "{col}") FROM {table_name}'
        cursor.execute(query)
        counts[col] = cursor.fetchone()[0]
    return counts

In [34]:
def compare_tables():
    config = load_config()
    log_path = create_log_file()
    logger = Logger(log_path)

    with connect_to_snowflake(config) as conn:
        with conn.cursor() as cursor:
            cols_1 = get_columns(cursor, TABLE_1, EXCLUDED_COLUMNS)
            cols_2 = get_columns(cursor, TABLE_2, EXCLUDED_COLUMNS)

            set_1 = set(cols_1)
            set_2 = set(cols_2)

            only_in_1 = sorted(set_1 - set_2)
            only_in_2 = sorted(set_2 - set_1)
            common = sorted(set_1 & set_2)

            logger.log(f"\n🔍 Comparing tables:")
            logger.log(f"    • Table 1: {TABLE_1}")
            logger.log(f"    • Table 2: {TABLE_2}\n")

            logger.log(f"📌 Columns only in {TABLE_1}: {only_in_1 or 'None'}")
            logger.log(f"📌 Columns only in {TABLE_2}: {only_in_2 or 'None'}\n")

            logger.log(f"📊 Distinct value comparison for {len(common)} common columns:\n")
            counts_1 = get_distinct_counts(cursor, TABLE_1, common)
            counts_2 = get_distinct_counts(cursor, TABLE_2, common)

            mismatches = []
            for col in common:
                if counts_1[col] != counts_2[col]:
                    mismatches.append((col, counts_1[col], counts_2[col]))

            for col, c1, c2 in mismatches:
                logger.log(f"⚠️  Column '{col}': {TABLE_1} has {c1}, {TABLE_2} has {c2}")

            num_ok = len(common) - len(mismatches)
            logger.log(f"\n✅ Remaining {num_ok} columns have matching distinct counts.\n")
            logger.log(f"📝 Log saved to {log_path}")

    logger.close()

In [35]:
compare_tables()

 pip install snowflake-connector-python[secure-local-storage]


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://myapps.wexinc.com/app/snowflake/exkaea789nRrm6HV7357/sso/saml?SAMLRequest=pZNbb%2BIwEIX%2FSuR9TuyEAK1FqGhRt5G6XcqlSLyZZACLxA4eh8C%2FX3NbdR%2Fal31znDP%2Bzsyxew%2BHsvD2YFBqlZAwYMQDlelcqnVCZtNn%2F454aIXKRaEVJOQISB76PRRlUfFBbTdqDLsa0HruIIX89CMhtVFcC5TIlSgBuc34ZPDrlUcB4wIRjHU4ci3JUTrWxtqKU9o0TdC0Am3WNGKMUXZPneok%2BUE%2BIarvGZXRVme6uJUcXE9fIELK4hPCKRxhdC18lOoygu8oy4sI%2Bct0OvJHvydT4g1u3T1phXUJZgJmLzOYjV8vBtA5WO4E697FLKjRb9zs%2FCiojNwLC4VU2wCVblaF2EKmy6q2DhG4FV1BTgu9lm5w6TAh1VbmM%2Fv%2BvpnDmlUtvXtLF4Dz%2BjHsbNPRcL%2BojnEtzGG%2BjGH9M86I93GLOTrFnCLWkKpTuNZtsajts44ftaZhzOMWD%2BOg3WUL4g2dQamEPVfeOiiPoqowaOAgVXZ2577pX%2BMUDlsBont3r8am7Lx8dFvtLkXU9BQeudwffnZg%2Bv81lR79fNT1Xr65qNLhSBcyO3rP2pTCfp1kGITnHZn7q7OUQylkMchzA4gu0aLQzZMB5yMh1tRAaP9C%2FfcB9

 pip install snowflake-connector-python[secure-local-storage]



🔍 Comparing tables:
    • Table 1: FIVETRAN_RAW.SCOTTS.SCOTTS
    • Table 2: _CLONE_DATAOPS_PENG_8343_SNOWPIPE_RAW.SCOTTS_PIPE.SCOTTS

📌 Columns only in FIVETRAN_RAW.SCOTTS.SCOTTS: ['EMPLOYMENT', 'HEAD_OFFICE_STATUS', 'HO_ADDRESS_1', 'HO_ADDRESS_2', 'HO_CITY', 'HO_COUNTRY', 'HO_NAME', 'HO_POSTAL_CODE', 'HO_PROVINCE', 'LOCATION_ADDRESS_1', 'LOCATION_ADDRESS_2', 'PARENT_COMPANY_ADDRESS_1', 'PARENT_COMPANY_ADDRESS_2', 'SCOTTS_ID', 'SCOTT_S_ID', 'WEB_SITE']
📌 Columns only in _CLONE_DATAOPS_PENG_8343_SNOWPIPE_RAW.SCOTTS_PIPE.SCOTTS: ['SCOTTSID']

📊 Distinct value comparison for 118 common columns:

⚠️  Column 'BUSINESS_TYPE_2': FIVETRAN_RAW.SCOTTS.SCOTTS has 25, _CLONE_DATAOPS_PENG_8343_SNOWPIPE_RAW.SCOTTS_PIPE.SCOTTS has 26
⚠️  Column 'BUSINESS_TYPE_3': FIVETRAN_RAW.SCOTTS.SCOTTS has 18, _CLONE_DATAOPS_PENG_8343_SNOWPIPE_RAW.SCOTTS_PIPE.SCOTTS has 21
⚠️  Column 'EMPLOYEE_COUNT': FIVETRAN_RAW.SCOTTS.SCOTTS has 555, _CLONE_DATAOPS_PENG_8343_SNOWPIPE_RAW.SCOTTS_PIPE.SCOTTS has 556
⚠️  Column